In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['recognizing-faces-in-the-wild', 'facenetkeras']


In [2]:
from collections import defaultdict
from glob import glob
from random import choice, sample
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Reshape, Dot, Flatten
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


In [3]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [4]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-3_77elv7
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-3_77elv7
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0si1bp8/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [5]:
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

In [6]:
import os
os.mkdir('/kaggle/working/train2')
!unzip ../input/recognizing-faces-in-the-wild/train.zip -d ./train2

Archive:  ../input/recognizing-faces-in-the-wild/train.zip
   creating: ./train2/F0002/
   creating: ./train2/F0005/
   creating: ./train2/F0009/
   creating: ./train2/F0010/
   creating: ./train2/F0016/
   creating: ./train2/F0017/
   creating: ./train2/F0020/
   creating: ./train2/F0022/
   creating: ./train2/F0023/
   creating: ./train2/F0024/
   creating: ./train2/F0030/
   creating: ./train2/F0031/
   creating: ./train2/F0036/
   creating: ./train2/F0038/
   creating: ./train2/F0039/
   creating: ./train2/F0040/
   creating: ./train2/F0041/
   creating: ./train2/F0044/
   creating: ./train2/F0049/
   creating: ./train2/F0050/
   creating: ./train2/F0051/
   creating: ./train2/F0052/
   creating: ./train2/F0054/
   creating: ./train2/F0058/
   creating: ./train2/F0059/
   creating: ./train2/F0063/
   creating: ./train2/F0064/
   creating: ./train2/F0068/
   creating: ./train2/F0070/
   creating: ./train2/F0074/
   creating: ./train2/F0077/
   creating: ./train2/F0081/
   creating: 

In [7]:
train_file_path = "../input/recognizing-faces-in-the-wild/train_relationships.csv"
train_folders_path = "./train2/"
val_famillies = "F09"

all_images = glob(train_folders_path + "*/*/*.jpg")

train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [8]:
import os
print('CURRENT DIR:', os.getcwd())
print('LEN IMAGES: ', len(all_images))

CURRENT DIR: /kaggle/working
LEN IMAGES:  12379


In [9]:
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]


In [10]:
# function for facenet
# def prewhiten(x):
#     if x.ndim == 4:
#         axis = (1, 2, 3)
#         size = x[0].size
#     elif x.ndim == 3:
#         axis = (0, 1, 2)
#         size = x.size
#     else:
#         raise ValueError('Dimension should be 3 or 4')

#     mean = np.mean(x, axis=axis, keepdims=True)
#     std = np.std(x, axis=axis, keepdims=True)
#     std_adj = np.maximum(std, 1.0/np.sqrt(size))
#     y = (x - mean) / std_adj
#     return y

# def read_img_fn(path):
#     img = cv2.imread(path)
#     img = cv2.resize(img,(197, 197))
#     img = np.array(img).astype(np.float)
#     return prewhiten(img)

### training

In [11]:
import keras
def read_img(path):
    img = image.load_img(path, target_size=(197, 197))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels


def baseline_model():
    input_1 = Input(shape=(197, 197, 3))
    input_2 = Input(shape=(197, 197, 3))

    base_model = VGGFace(model='resnet50', include_top=False)
#     base_model = keras.applications.InceptionResNetV2(
#             include_top=False, weights="imagenet", input_shape=(197, 197, 3))
    
    # facenet
#     model_path = '../input/facenetkeras/facenet_keras.h5'
#     base_model = keras.models.load_model(model_path)

    for x in base_model.layers[:-3]:
        x.trainable = True
    
    x1 = base_model(input_1)
    x2 = base_model(input_2)
    
#     x1 = Reshape((1, 1 ,128))(x1)
#     x2 = Reshape((1, 1 ,128))(x2)

    # dot product
#     x1_ = Concatenate(axis=1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
#     x2_ = Concatenate(axis=1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])
#     x1_ = Reshape(target_shape=(2,2048))(x1_)
#     x2_ = Reshape(target_shape=(2,2048))(x2_)

#     x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
#     x = Flatten()(x_dot)

    # (x1-x2)^2 concat (x1^2 - x2^2)
    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x = Concatenate(axis=-1)([x4, x3])

#     # dot product (bad accuracy)
#     x = Dot(axes=1, normalize=True)([x1,x2])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.01)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc', auc], optimizer=Adam(0.00001))

    model.summary()

    return model


In [12]:
file_path = "resnet50.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

csv_logger = CSVLogger('training.log')

callbacks_list = [checkpoint, reduce_on_plateau, csv_logger]

model = baseline_model()
#model.load_weights(file_path)
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=100, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)


94699520/94694792 [==============================] - 4s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 2048)

/opt/conda/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


 - 87s - loss: 3.6220 - acc: 0.5884 - auc: 0.6123 - val_loss: 3.6755 - val_acc: 0.5531 - val_auc: 0.5744

Epoch 00001: val_acc improved from -inf to 0.55312, saving model to resnet50.h5
Epoch 2/100
 - 62s - loss: 1.7011 - acc: 0.6231 - auc: 0.6805 - val_loss: 2.0548 - val_acc: 0.5900 - val_auc: 0.6129

Epoch 00002: val_acc improved from 0.55312 to 0.59000, saving model to resnet50.h5
Epoch 3/100
 - 62s - loss: 0.9565 - acc: 0.6566 - auc: 0.7113 - val_loss: 1.2499 - val_acc: 0.6188 - val_auc: 0.6538

Epoch 00003: val_acc improved from 0.59000 to 0.61875, saving model to resnet50.h5
Epoch 4/100
 - 62s - loss: 0.7414 - acc: 0.6641 - auc: 0.7315 - val_loss: 0.9400 - val_acc: 0.6500 - val_auc: 0.6949

Epoch 00004: val_acc improved from 0.61875 to 0.65000, saving model to resnet50.h5
Epoch 5/100
 - 61s - loss: 0.6311 - acc: 0.6900 - auc: 0.7566 - val_loss: 0.8690 - val_acc: 0.6500 - val_auc: 0.7127

Epoch 00005: val_acc did not improve from 0.65000
Epoch 6/100
 - 63s - loss: 0.5740 - acc: 0.

### prediction

In [13]:
# print(os.listdir("../input"))

In [14]:
import os
# os.mkdir('/kaggle/working/test2')
# !unzip ../input/test.zip -d ./test2

In [15]:
def read_img(path):
    img = image.load_img(path, target_size=(197, 197))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [16]:
# test_path = "./test2/"

# import keras
# model = keras.models.load_model('./vgg_face.h5')

# def chunker(seq, size=32):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))


# from tqdm import tqdm

# submission = pd.read_csv('../input/sample_submission.csv') #/recognizing-faces-in-the-wild

# predictions = []

# for batch in tqdm(chunker(submission.img_pair.values)):
#     X1 = [x.split("-")[0] for x in batch]
#     X1 = np.array([read_img(test_path + x) for x in X1])

#     X2 = [x.split("-")[1] for x in batch]
#     X2 = np.array([read_img(test_path + x) for x in X2])

#     pred = model.predict([X1, X2]).ravel().tolist()
#     predictions += pred

# submission['is_related'] = predictions

# submission.to_csv("vgg_face.csv", index=False)

In [17]:

# test_path = "../input/test/"


# def chunker(seq, size=32):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))


# from tqdm import tqdm

# submission = pd.read_csv('../input/sample_submission.csv')

# predictions = []

# for batch in tqdm(chunker(submission.img_pair.values)):
#     X1 = [x.split("-")[0] for x in batch]
#     X1 = np.array([read_img(test_path + x) for x in X1])

#     X2 = [x.split("-")[1] for x in batch]
#     X2 = np.array([read_img(test_path + x) for x in X2])

#     pred = model.predict([X1, X2]).ravel().tolist()
#     predictions += pred

# submission['is_related'] = predictions

# submission.to_csv("vgg_face.csv", index=False)


## REFERENCE:
[Kind of solutions](https://www.kaggle.com/c/recognizing-faces-in-the-wild/discussion/93263#latest-547742)   
[Code sharing - 0.81 LB](https://www.kaggle.com/c/recognizing-faces-in-the-wild/discussion/92454#latest-547744)
